In [1]:
import csv, emoji
import datetime
import itertools
import os
import pickle
import random
import re
import time
from sklearn.metrics import classification_report
# import ipywidgets as widgets
import openai
import pandas as pd
import torch
from dotenv import load_dotenv
from torch.utils.data import DataLoader, RandomSampler
from sklearn.model_selection import KFold
import json

openai.api_key = os.getenv("YOUR_OPEN_AI_API_KEY")

from prompt_template import create_prompt

In [2]:
'''
In this block of code I am cleaning the data as khouloud did
'''
class TextProcessor:
    def __init__(self):
        pass
    
    def preprocess_tweet_text(self, text):
        text = re.sub(r'#([^ ]*)', r'\1', text)
        text = re.sub(r'https.*[^ ]', 'URL', text)
        text = re.sub(r'http.*[^ ]', 'URL', text)
        text = emoji.demojize(text)
        text = re.sub(r'(:.*?:)', r' \1 ', text)
        text = re.sub(' +', ' ', text)
        text = text.strip()
        return text
    
    def preprocess_dataframe(self, df):
        df['text'] = df['text'].apply(self.preprocess_tweet_text)
        return df

    
# Instantiate a TextProcessor object
text_processor = TextProcessor()

In [ ]:
'''
reading the csv file in pandas dataframe and converting the label
column to string column

Reason of choosing that string label:
I believe there should be a strong correlation between what the label is 
when we give it to the GPT model, it must have some prior information about 
the label.

To Test this I have to repeat the experiment where I will ask the model to 
label as 0 or 1 (not hate speech and hate speech respectively)


'''

df_hateval_test= pd.read_csv(expanded_path_hateval_test_csv)
print("==Before Converting Numerical Label to String===")
print(df_hateval_test.head())
print("==After Converting Numerical Label to String===")
# %debug
# Define a dictionary to map values
mapping = {1: "Hate speech", 0: "Not hate speech"}
df_hateval_test['HS'] = df_hateval_test['HS'].map(mapping)
print(df_hateval_test.head())
print("hello darkness my old friend")
print(f"Lenght of the dataframe: {len(df_hateval_test)}")

In [ ]:
finalDF = df_hateval_test
# folds = KFold(n_splits=5)

# creating a dictionary to save the classification report.
cv_dictionary = dict()

# it means the same as run and it is used  for the purpose
# for printing and saving all the importante report w.r.t the current run
fold = 2

# defines how many iteration you want to do for the current experiment.
total_iteration = 3

for i in range(total_iteration):
    
    fold += 1
    print(f"===========RUN: {fold} starting===========")
    # Cleaning the test dataset
    df_test = text_processor.preprocess_dataframe(df_hateval_test)
    
    # this will take only 5 instances from the test set and perform the model inference
    # df_test = df_test.loc[:5-1]

    '''
    getting the batches of batches as it was before
    '''
    # create an empty list to store the index values
    batches_of_batches = []

    # loop over the dataframe using iterrows() method
    for index, row in df_test.iterrows():
        # append the index value to the list
        batches_of_batches.append([index])
        

    if not os.path.exists(f'response_hatespeech_run_{fold}.csv'):
        with open(f"response_hatespeech_run_{fold}.csv", "a", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Input", "Complete Response", "Model Output", "Date and Time"])

    if not os.path.exists(f'groundtruth_hatespeech_run_{fold}.csv'):
        with open(f"groundtruth_hatespeech_run_{fold}.csv", "a", newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["test-instance-index","groundtruth-text", "groundtruth-label"])


    # Go through each batch (it has n instances)
    for batch_number, i in enumerate(batches_of_batches):

        text_list_gold = list()


        # loop through each instances within a batch
        for k in i:

            # k = index

            text_list_gold.append(df_test.loc[k]['text'])
            data = [k, df_test.loc[k]['text'], df_test.loc[k]['HS']]

            # Open the CSV file for writing the ground truth
            with open(f"groundtruth_hatespeech_run_{fold}.csv", "a", newline='', encoding="utf-8") as file:
                writer = csv.writer(file)
                # Write the data rows
                writer.writerow(data)

        # Below creating the final template, containing the appended text and the examples that 
        # are to be classified by the model

        # text_list_gold has the text that I want to classify with the model
        for text in text_list_gold:
            # template += f"\nText:{text}\nLabel:"
            template = create_prompt(text, fourshot=True)
        # print(template)

        # break

        # Note: 

        '''
        In all experiments throughout this paper, for our calls to GPT-3,
        we Players_(film)Players_(film)set the temperature to 0.0 and the top_P to 1.0. 1 These settings
        serve two purposes. First, they ensure reproducibility, specifically
        that GPT-3 will always have the same response given the prompt
        we pass it. Second, they minimize the risk that GPT-3 will wander off topic or hallucinate.

        Paper: Can GPT-3 Perform Statutory Reasoning?
        Link: https://arxiv.org/pdf/2302.06100.pdf

        '''

        retry_limit = 10  # Maximum number of retries

        try_count = 0

        while try_count < retry_limit:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": "You must reply with either 'Hate speech' or 'Not hate speech' based solely on the linguistic content of the text that you will be asked to classify."},
                        {"role": "user", "content": template}
                    ],
                    temperature=0,
                )

                # If the response is successful, break out of the loop
                break

            except openai.error.RateLimitError as e:
                print("Rate limit exceeded. Waiting for 3 minute before retrying...")
                time.sleep(180)  # Wait for 3 minute (adjust as needed)
                try_count += 1  # Increment the try count and retry

            except Exception as e:
                # catch me if you can i.e. catch everything. I am determined to get the model output no matter what - we live only once right?
                print(f"This Exception: {e}")
                print(f"For this batch, we got the error: {batch_number}")
                print(f"These are the instances that the model could not create a completion for: {i}")
                print(f"For this exception {e}, Waiting for 5 minute before retrying...")
                time.sleep(300)  # Wait for 3 minute (adjust as needed)
                try_count += 1

        if try_count == retry_limit:
            print("Reached the maximum number of retries. Unable to get a successful response.\nTherefore using the previous label as the current label for the test text")

        # print("something")
        # saving some data
        now = datetime.datetime.now()
        data = [template, response, response['choices'][0]['message']['content'], now.strftime("%Y-%m-%d %H:%M:%S")]
        # Open the CSV file for writing
        with open(f"response_hatespeech_run_{fold}.csv", "a", newline='',encoding="utf-8") as file:
            writer = csv.writer(file)
            # Write the data rows
            writer.writerow(data)
        
        label = response['choices'][0]['message']['content']
        # i[0] is the instance index that is coming from batches_of_batches
        instance_label = [(i[0], label)]
        try:
            with open(f"modeloutput_hatespeech_run_{fold}.csv", 'x') as file:
                # Create a new Dataframe with the label
                df = pd.DataFrame(instance_label, columns=["test-instance-index","gpt-label"])
                # Write the DataFrame to the csv file
                df.to_csv(f"modeloutput_hatespeech_run_{fold}.csv", index=False)
        except FileExistsError:
            # If the file already exists, append the data to it
            with open(f"modeloutput_hatespeech_run_{fold}.csv", 'a', encoding="utf-8") as file:
                # Create a new DataFrame with the data
                df = pd.DataFrame(instance_label, columns=["test-instance-index","gpt-label"])
                # Append the DataFrame to the csv file
                df.to_csv(file, header=False, index=False)

    
    # reading the models output and groundtruth for creating the classificaion report

    ground_truth = pd.read_csv(f"groundtruth_hatespeech_run_{fold}.csv")
    model_output = pd.read_csv(f"modeloutput_hatespeech_run_{fold}.csv")
    
    # Ground truth data
    y_true = ground_truth["groundtruth-label"]

    # Model output data
    y_pred = model_output["gpt-label"]


    # Generate classification report
    report = classification_report(y_true, y_pred)
    
    report_dictionary = classification_report(y_true, y_pred, output_dict=True)
    #saving each classificaiton report for the current run
    cv_dictionary[f'run_{fold}'] = report_dictionary
    
    print(report)
    print("\n")
    print(f"=========Run: {fold} Finish ==============\n")
    print(f"\n=====Sleeping for 5 minute before going to the next run=====\n")
    time.sleep(300)


# saving the complete report for each run 
with open("cv_run_3_4_5_report.json", "w") as f:
          json.dump(cv_dictionary, f, indent=4)  

How to handle rate limits: 
https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

# Extra Code

In [ ]:
'''
For string output
'''

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        # {"role": "system", "content": "You must reply with either only 'Hate speech' or 'Not hate speech' based solely on the text that you will be asked to classify"}
        {"role": "user", "content": template_temp}

    ],
    temperature=0,
)
print(response['choices'][0]['message']['content'])